In [ ]:
!pip install skforecast tqdm catboost

In [10]:
import pandas as pd
from tqdm import tqdm
import lightgbm
import sklearn
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
import skforecast
from sklearn.feature_selection import RFECV
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import bayesian_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import select_features
import matplotlib.pyplot as plt
import numpy as np
import os
from os import path
import shutil
import re
import traceback
%matplotlib inline

In [11]:
pipeline_categorical = make_pipeline(
                           OrdinalEncoder(
                               dtype=int,
                               handle_unknown="use_encoded_value",
                               unknown_value=-1,
                               encoded_missing_value=-1
                           ),
                           FunctionTransformer(
                               func=lambda x: x.astype('category'),
                               feature_names_out= 'one-to-one'
                           )
                       )

transformer_exog = make_column_transformer(
                       (
                           pipeline_categorical,
                           make_column_selector(dtype_exclude=np.number)
                       ),
                       remainder="passthrough",
                       verbose_feature_names_out=False,
                   ).set_output(transform="pandas")

# Lags grid
lags_grid = tuple([12, 24, [1, 2, 3, 4, 7, 9, 24]])

# Regressor hyperparameters search space for lightgbm
def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 400, 1200, step=100),
        'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 25, 500),
        'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 0.5),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1, step=0.1),
        'max_bin'         : trial.suggest_int('max_bin', 50, 250, step=25),
        'reg_alpha'       : trial.suggest_float('reg_alpha', 0, 1, step=0.1),
        'reg_lambda'      : trial.suggest_float('reg_lambda', 0, 1, step=0.1),
        'lags'            : trial.suggest_categorical('lags', lags_grid)
    }
    return search_space

In [12]:
def search_hyperparameters(data, end_train, end_valid, exog_features,transformer_exog):

  # instantiate a forcaster transformer with categorical features
    forecaster = ForecasterAutoreg(
        regressor = LGBMRegressor(random_state=15926, verbose=-1),
        lags = 24,
        transformer_exog = transformer_exog,
        fit_kwargs = {"categorical_feature": "auto"}
    )

    # search for best parameters
    results_search, frozen_trial = bayesian_search_forecaster(
    forecaster         = forecaster,
    y                  = data.loc[:end_valid, 'gw-level'],
    exog               = data.loc[:end_valid, exog_features],
    search_space       = search_space,
    steps              = 30,
    refit              = False,
    metric             = 'mean_absolute_percentage_error',
    initial_train_size = len(data.loc[:end_train]),
    fixed_train_size   = False,
    n_trials           = 20,
    random_state       = 123,
    return_best        = True,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
    )

    best_params = results_search['params'].iat[0]

    return best_params

def train_and_predict(data, best_params, actual_data, end_valid, end_train, valid_num, train_num, df_idx, exog_features, transformer_exog):

  # train for evaluation of the model
  forecaster = ForecasterAutoreg(
  regressor          = LGBMRegressor(**best_params, random_state=15926, verbose=-1),
  lags               = 24,
  transformer_exog   = transformer_exog,
  fit_kwargs         = {"categorical_feature": "auto"}
  )
  # train the model the time series train and validation dataset
  forecaster.fit(
    y    = data.loc[:end_train, 'gw-level'],
    exog = data.loc[:end_train, exog_features]
  )

  # make predictions and evalute the model
  predictions = forecaster.predict(
      exog     = data.loc[df_idx[train_num+1]:, exog_features],
      steps    = 26
  )
  df_preds = pd.DataFrame(predictions)
  preds = df_preds["pred"].values
  # evaluate on symmetric mean absolute percentage error
  smape_value = smape(actual_data, preds)

  # train for future predictions
  forecaster = ForecasterAutoreg(
  regressor          = LGBMRegressor(**best_params, random_state=15926, verbose=-1),
  lags               = 24,
  transformer_exog   = transformer_exog, #one_hot_encoder,
  fit_kwargs         = {"categorical_feature": "auto"}
  )
  # train the model the time series train and validation dataset
  forecaster.fit(
    y    = data.loc[:end_valid, 'gw-level'],
    exog = data.loc[:end_valid, exog_features]
  )

  # make predictions into the future
  predictions = forecaster.predict(
    exog     = data.loc[df_idx[valid_num+1]:, exog_features],
    steps    = 26
  )
  df_preds = pd.DataFrame(predictions)

  # free resources since it's going to run on iterations
  del forecaster

  return df_preds, smape_value


In [13]:
# Define the function to return the SMAPE value
def smape(A, F):
    tmp = 2 * np.abs(F - A) / (np.abs(A) + np.abs(F))
    len_ = np.count_nonzero(~np.isnan(tmp))
    if len_ == 0 and np.nansum(tmp) == 0: # Deals with a special case
        return 100
    return round(100 / len_ * np.nansum(tmp), 3)

In [18]:
def populate_test_data(data_dir):

  preds_dict = {}
  smape_dict = {}


  # collect all files in the directory
  filenames = os.listdir(data_dir)
  filenames = filenames[:20]

  try:

    for filename in tqdm(filenames):

        hrbnz01 = filename.split(".")[0].split("-")[-1]
        filepath = path.join(data_dir, filename)
        df_exog = pd.read_csv(filepath)
        df_exog["season"] = df_exog["season"].astype("category")
        df_exog["weather"] = df_exog["weather"].astype("category")
        df_exog["date"] = pd.to_datetime(df_exog["date"])
        df_exog.set_index("date", inplace=True)
        df_exog.index = pd.date_range(start=df_exog.index.min(), end=df_exog.index.max(), freq='MS')

        # get the estimate end train and end validation dates
        data = df_exog.copy()
        exog_data = data.drop("gw-level", axis=1)
        exog_features = exog_data.columns
        df_idx = data.index
        train_num = int(len(data) * 0.8)
        valid_num = len(data.loc[:"2021-11-01"])
        end_train = df_idx[train_num]
        end_valid = df_idx[valid_num]
        end_evaluation = df_idx[train_num+26]
        evaluate_data = data.loc[df_idx[train_num+1]: end_evaluation, "gw-level"].values


        # tune for best hyperparamters and evaluate on MAPE metric
        best_params = search_hyperparameters(data, end_train, end_valid, exog_features, transformer_exog)

        # train and make predict into 26 months in the future of the test template
        df_predictions, smape = train_and_predict(data,best_params, evaluate_data, end_valid, end_train, valid_num, train_num, df_idx, exog_features, transformer_exog, hrbnz01)
        df_predictions["pred"] =  df_predictions["pred"].round(2)
        preds_dict[hrbnz01] = df_predictions['pred'].values
        smape_dict[hrbnz01] = smape


  except Exception as ex:
    print("[Error]")
    print(traceback.format_exc())

  df_smape = pd.DataFrame(smape_dict, index=[0])
  df_final_preds = pd.DataFrame(preds_dict, index=range(26))
  print("> Done")

  return df_final_preds, df_smape

In [ ]:
processed_data_dir = "/content/drive/MyDrive/processed_data_part1"
#test_template = "/content/drive/MyDrive/gw_test_empty.csv"

df_submission_lgbm, df_smape_lgbm = populate_test_data(processed_data_dir)
# df_submission.to_csv("df_submission.csv", index=False)
# df_smape.to_csv("smape_score.csv", index=False)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1, 2, 3, 4, 7, 9, 24] which is of type list.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_discretization.py:245: UserWarning: Feature 0 is constant and will be replaced with 0.
  warnings.warn(
  5%|▌         | 1/20 [00:04<01:21,  4.26s/it]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12] 
  Parameters: {'n_estimators': 600, 'max_depth': 9, 'min_data_in_leaf': 408, 'learning_rate': 0.05151409102341216, 'feature_fraction': 1.0, 'max_bin': 250, 'reg_alpha': 0.4, 'reg_lambda': 0.1}
  Backtesting metric: 0.0028209927694297756



  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1, 2, 3, 4, 7, 9, 24] which is of type list.
  warnings.warn(message)


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  7  9 24] 
  Parameters: {'n_estimators': 1100, 'max_depth': 3, 'min_data_in_leaf': 87, 'learning_rate': 0.24203460874368538, 'feature_fraction': 0.9, 'max_bin': 225, 'reg_alpha': 0.8, 'reg_lambda': 0.4}
  Backtesting metric: 0.0028786271903392022



100%|██████████| 20/20 [01:13<00:00,  3.69s/it]

> Done


In [ ]:
df_smape_lgbm.index = pd.Index(['smape'])
print(f" average smape: {round(df_smape_lgbm.mean(axis=1)[0],4)}")
df_smape_lgbm


 average smape: 0.1377


,300111,345710,329078,301838,345116,345512,326934,321646,335208,309625,309948,328104,310029,335778,326868,306167,345314,313569,313387,331124
smape,0.313,0.026,0.13,0.072,0.082,0.235,0.126,0.03,0.171,0.088,0.049,0.064,0.07,0.046,0.058,0.447,0.193,0.042,0.093,0.419


In [ ]:
df_submission_lgbm.index = pd.date_range(start=pd.Timestamp("2021-12-01") + pd.DateOffset(months=1), periods=26, freq='MS')
df_submission_lgbm

,300111,345710,329078,301838,345116,345512,326934,321646,335208,309625,309948,328104,310029,335778,326868,306167,345314,313569,313387,331124
2022-01-01,153.89,399.97,238.26,200.74,224.31,230.86,242.54,510.75,328.43,492.98,638.33,424.73,658.55,251.62,283.86,113.63,120.67,181.15,144.05,140.06
2022-02-01,153.89,400.00,238.26,200.71,224.38,230.76,242.46,510.81,328.39,493.27,638.32,424.62,658.54,251.68,283.86,113.63,120.68,181.15,144.07,140.29
2022-03-01,153.99,399.97,238.26,200.72,224.33,230.61,242.53,510.88,328.38,493.44,638.42,424.62,658.59,251.64,283.86,113.63,120.74,181.15,144.08,140.68
2022-04-01,154.02,399.99,238.26,200.68,224.22,230.51,242.84,511.05,328.16,493.66,638.72,424.64,658.59,251.61,283.86,113.63,120.83,181.15,144.08,140.48
2022-05-01,154.00,400.00,238.26,200.66,224.40,230.80,243.14,511.18,328.39,493.41,638.95,424.60,658.53,251.63,283.86,113.63,120.87,181.15,144.08,140.52
2022-06-01,153.95,400.08,238.26,200.62,224.50,230.90,243.35,511.21,328.42,492.76,639.07,424.59,658.55,251.69,283.86,113.63,120.92,181.15,144.07,140.40
2022-07-01,153.86,400.16,238.26,200.64,224.60,230.99,243.30,511.08,328.54,492.57,639.08,424.60,658.67,251.68,283.86,113.63,120.92,181.15,144.09,139.75
2022-08-01,153.86,400.16,238.26,200.63,224.56,230.98,243.28,511.01,328.89,492.93,639.02,424.60,658.81,251.65,283.86,113.63,120.82,181.15,144.10,140.42
2022-09-01,153.85,400.16,238.26,200.65,224.50,230.91,243.18,510.93,329.08,493.20,639.02,424.61,658.95,251.74,283.86,113.63,120.71,181.15,144.09,140.08
2022-10-01,153.76,400.16,238.26,200.70,224.51,231.07,242.53,510.87,329.16,493.27,638.91,424.60,658.96,251.74,283.86,113.63,120.64,181.15,144.11,139.69


## Trying XGBoost Model for Comparison

In [ ]:
# Regressor hyperparameters search space for xgboost
def search_space(trial):

  # Lags grid
  lags_grid = tuple([12, [1, 2, 3, 4, 7, 9, 12]])

  search_space  = {
      'n_estimators'    : trial.suggest_int('n_estimators', 400, 1200, step=100),
      'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
      'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 1),
      'subsample'       : trial.suggest_float('subsample', 0.1, 1),
      'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
      'gamma'           : trial.suggest_float('gamma', 0, 1),
      'reg_alpha'       : trial.suggest_float('reg_alpha', 0, 1),
      'reg_lambda'      : trial.suggest_float('reg_lambda', 0, 1),
      'lags'            : trial.suggest_categorical('lags', lags_grid)
  }
  return search_space

In [ ]:
def search_hyperparameters(data, end_train, end_valid, exog_features,transformer_exog):

  # instantiate a forcaster transformer with categorical features
  forecaster = ForecasterAutoreg(
  regressor = XGBRegressor(
                  tree_method = 'hist',
                  enable_categorical = True,
                  random_state = 123
              ),
  lags = 24,
  transformer_exog = transformer_exog
  )

  # search for best parameters
  results_search, frozen_trial = bayesian_search_forecaster(
  forecaster         = forecaster,
  y                  = data.loc[:end_valid, 'gw-level'],
  exog               = data.loc[:end_valid, exog_features],
  search_space       = search_space,
  steps              = 30,
  refit              = False,
  metric             = 'mean_absolute_percentage_error',
  initial_train_size = len(data.loc[:end_train]),
  fixed_train_size   = False,
  n_trials           = 20,
  random_state       = 123,
  return_best        = True,
  n_jobs             = 'auto',
  verbose            = False,
  show_progress      = True
  )

  best_params = results_search['params'].iat[0]

  return best_params

def train_and_predict(data, best_params, actual_data, end_valid, end_train, valid_num, train_num, df_idx, exog_features, transformer_exog, hrbnz01):

  # train for evaluation of the model
  forecaster = ForecasterAutoreg(
  regressor = XGBRegressor(**best_params,
                  tree_method = 'hist',
                  enable_categorical = True,
                  random_state = 123
              ),
  lags = 24,
  transformer_exog = transformer_exog
  )

  # train the model the time series train and validation dataset
  forecaster.fit(
    y    = data.loc[:end_train, 'gw-level'],
    exog = data.loc[:end_train, exog_features]
  )

  # make predictions and evalute the model
  predictions = forecaster.predict(
      exog     = data.loc[df_idx[train_num+1]:, exog_features],
      steps    = 26
  )
  df_preds = pd.DataFrame(predictions)
  preds = df_preds["pred"].values
  # evaluate on symmetric mean absolute percentage error
  smape_value = smape(actual_data, preds)

  # train for future predictions
  forecaster = ForecasterAutoreg(
  regressor = XGBRegressor(**best_params,
                  tree_method = 'hist',
                  enable_categorical = True,
                  random_state = 123
              ),
  lags = 24,
  transformer_exog = transformer_exog
  )
  # train the model the time series train and validation dataset
  forecaster.fit(
    y    = data.loc[:end_valid, 'gw-level'],
    exog = data.loc[:end_valid, exog_features]
  )

  # make predictions into the future
  predictions = forecaster.predict(
    exog     = data.loc[df_idx[valid_num+1]:, exog_features],
    steps    = 26
  )
  df_preds = pd.DataFrame(predictions)

  # free resources since it's going to run on iterations
  del forecaster

  return df_preds, smape_value

In [ ]:
processed_data_dir = "/content/drive/MyDrive/processed_data_part1"
#test_template = "/content/drive/MyDrive/gw_test_empty.csv"

df_submission_xgb, df_smape_xgb = populate_test_data(processed_data_dir)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1, 2, 3, 4, 7, 9, 12] which is of type list.
  warnings.warn(message)


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12] 
  Parameters: {'n_estimators': 600, 'max_depth': 5, 'learning_rate': 0.03885337722243354, 'subsample': 0.3604262920151898, 'colsample_bytree': 0.9501979211086962, 'gamma': 0.994411218105991, 'reg_alpha': 0.003677635938920898, 'reg_lambda': 0.47886494132474805}
  Backtesting metric: 0.0024727797496551145



  5%|▌         | 1/20 [00:09<03:06,  9.80s/it]

...

  0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1, 2, 3, 4, 7, 9, 12] which is of type list.
  warnings.warn(message)


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12] 
  Parameters: {'n_estimators': 400, 'max_depth': 5, 'learning_rate': 0.115434619925389, 'subsample': 0.7636067026603856, 'colsample_bytree': 0.3513624942452013, 'gamma': 0.49302435382809684, 'reg_alpha': 0.1860900286451497, 'reg_lambda': 0.7240813648907406}
  Backtesting metric: 0.003065588118081405



100%|██████████| 20/20 [02:49<00:00,  8.47s/it]

> Done


In [ ]:
df_smape_xgb.index = pd.Index(['smape'])
print(f" average smape: {round(df_smape_xgb.mean(axis=1)[0],4)}")
df_smape_xgb

 average smape: 0.1213


,300111,345710,329078,301838,345116,345512,326934,321646,335208,309625,309948,328104,310029,335778,326868,306167,345314,313569,313387,331124
smape,0.107,0.022,0.138,0.064,0.095,0.243,0.14,0.037,0.096,0.063,0.044,0.067,0.057,0.048,0.07,0.443,0.218,0.038,0.114,0.322


In [ ]:
df_submission_xgb.index = pd.date_range(start=pd.Timestamp("2021-12-01") + pd.DateOffset(months=1), periods=26, freq='MS')
df_submission_xgb

,300111,345710,329078,301838,345116,345512,326934,321646,335208,309625,309948,328104,310029,335778,326868,306167,345314,313569,313387,331124
2022-01-01,153.93,400.01,238.10,200.75,224.48,230.60,242.35,510.77,328.43,492.36,638.41,424.80,658.66,251.77,283.82,113.31,120.81,181.16,143.99,140.20
2022-02-01,153.92,400.01,238.11,200.79,224.48,230.54,242.82,510.81,328.43,492.85,638.41,424.80,658.65,251.75,283.78,113.35,120.91,181.16,143.99,140.50
2022-03-01,153.92,400.01,238.11,200.84,224.48,230.62,242.48,510.98,328.43,493.47,638.44,424.80,658.69,251.75,283.73,113.41,120.99,181.16,143.99,140.62
2022-04-01,153.92,400.01,238.04,200.89,224.38,230.63,243.01,511.04,328.43,493.32,638.65,424.80,658.75,251.75,283.71,113.43,120.99,181.16,143.99,140.47
2022-05-01,153.91,400.03,238.04,200.84,224.54,230.67,243.43,511.14,328.36,493.19,638.90,424.80,658.75,251.75,283.68,113.41,120.95,181.16,143.99,140.63
2022-06-01,153.90,400.03,238.04,200.71,224.55,230.75,244.02,511.18,328.36,493.35,639.17,424.87,658.79,251.75,283.71,113.41,120.89,181.16,143.99,140.62
2022-07-01,153.83,400.06,238.04,200.65,224.56,230.65,243.81,511.11,328.26,493.01,639.11,424.80,658.89,251.75,283.81,113.39,120.71,181.16,143.99,140.52
2022-08-01,153.72,400.06,238.04,200.59,224.60,230.57,243.71,510.95,328.44,492.81,639.13,424.80,658.95,251.75,283.84,113.37,120.78,181.16,143.99,140.24
2022-09-01,153.64,400.03,238.04,200.63,224.54,230.80,243.67,510.91,328.44,493.05,639.11,424.80,658.94,251.75,283.84,113.37,120.71,181.16,143.99,140.04
2022-10-01,153.60,400.01,238.04,200.77,224.60,230.58,242.76,510.82,328.34,493.54,638.98,424.87,658.91,251.75,283.75,113.37,120.61,181.16,143.99,139.92


## HistGradientBoostingRegressor Model

#### I have only trained the new data and predict on HistGradientBoostingRegressor since it outperforms other models

In [15]:
# one-hot encoding
categorical_features = ["weather", "season"]
transformer_exog = make_column_transformer(
    (
        OrdinalEncoder(
            dtype=int,
            handle_unknown="use_encoded_value",
            unknown_value=-1,
            encoded_missing_value=-1
        ),
        categorical_features
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")


# Regressor hyperparameters search space
def search_space(trial):

  # Lags grid
  lags_grid = tuple([12, 24, [1, 2, 3, 4, 7, 9, 24]])

  search_space  = {
      'max_iter'          : trial.suggest_int('max_iter', 400, 1200, step=100),
      'max_depth'         : trial.suggest_int('max_depth', 3, 10, step=1),
      'learning_rate'     : trial.suggest_float('learning_rate', 0.01, 1),
      'min_samples_leaf'  : trial.suggest_int('min_samples_leaf', 1, 20, step=1),
      'l2_regularization' : trial.suggest_float('l2_regularization', 0, 1),
      'lags'              : trial.suggest_categorical('lags', lags_grid)
  }
  return search_space

In [19]:
def search_hyperparameters(data, end_train, end_valid, exog_features,transformer_exog):

  # instantiate a forcaster transformer with categorical features
  forecaster = ForecasterAutoreg(
  regressor = HistGradientBoostingRegressor(
                  categorical_features=categorical_features,
                  random_state=123
              ),
  lags = 24,
  transformer_exog = transformer_exog
  )

  # search for best parameters
  results_search, frozen_trial = bayesian_search_forecaster(
  forecaster         = forecaster,
  y                  = data.loc[:end_valid, 'gw-level'],
  exog               = data.loc[:end_valid, exog_features],
  search_space       = search_space,
  steps              = 30,
  refit              = False,
  metric             = 'mean_absolute_percentage_error',
  initial_train_size = len(data.loc[:end_train]),
  fixed_train_size   = False,
  n_trials           = 20,
  random_state       = 123,
  return_best        = True,
  n_jobs             = 'auto',
  verbose            = False,
  show_progress      = True
  )

  best_params = results_search['params'].iat[0]

  return best_params

def train_and_predict(data, best_params, actual_data, end_valid, end_train, valid_num, train_num, df_idx, exog_features, transformer_exog, hrbnz01):

  # train for evaluation of the model
  forecaster = ForecasterAutoreg(
  regressor = HistGradientBoostingRegressor(**best_params,
                  categorical_features=categorical_features,
                  random_state=123
              ),
  lags = 24,
  transformer_exog = transformer_exog
  )

  # train the model the time series train and validation dataset
  forecaster.fit(
    y    = data.loc[:end_train, 'gw-level'],
    exog = data.loc[:end_train, exog_features]
  )

  # make predictions and evalute the model
  predictions = forecaster.predict(
      exog     = data.loc[df_idx[train_num+1]:, exog_features],
      steps    = 26
  )
  df_preds = pd.DataFrame(predictions)
  preds = df_preds["pred"].values
  # evaluate on symmetric mean absolute percentage error
  smape_value = smape(actual_data, preds)

  # train for future predictions
  forecaster = ForecasterAutoreg(
  regressor = HistGradientBoostingRegressor(**best_params,
                  categorical_features=categorical_features,
                  random_state=123
              ),
  lags = 24,
  transformer_exog = transformer_exog
  )
  # train the model the time series train and validation dataset
  forecaster.fit(
    y    = data.loc[:end_valid, 'gw-level'],
    exog = data.loc[:end_valid, exog_features]
  )

  # make predictions into the future
  predictions = forecaster.predict(
    exog     = data.loc[df_idx[valid_num+1]:, exog_features],
    steps    = 26
  )
  df_preds = pd.DataFrame(predictions)

  # free resources since it's going to run on iterations
  del forecaster

  return df_preds, smape_value

In [20]:
processed_data_dir = "/content/drive/MyDrive/clean_processed_data_part1"

df_submission_hist, df_smape_hist = populate_test_data(processed_data_dir)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1, 2, 3, 4, 7, 9, 24] which is of type list.
  warnings.warn(message)


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
  Parameters: {'max_iter': 1000, 'max_depth': 5, 'learning_rate': 0.2345829390285611, 'min_samples_leaf': 12, 'l2_regularization': 0.7194689697855631}
  Backtesting metric: 0.000501016334235111



  5%|▌         | 1/20 [01:36<30:35, 96.58s/it]

...

  0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1, 2, 3, 4, 7, 9, 24] which is of type list.
  warnings.warn(message)


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
  Parameters: {'max_iter': 400, 'max_depth': 4, 'learning_rate': 0.47361554254833266, 'min_samples_leaf': 20, 'l2_regularization': 0.9207154485812868}
  Backtesting metric: 0.001259566662602886



100%|██████████| 20/20 [29:25<00:00, 88.26s/it]

> Done


In [22]:
df_smape_hist.index = pd.Index(['smape'])
print(f" average smape: {round(df_smape_hist.mean(axis=1)[0],4)}")
df_smape_hist

 average smape: 0.0982


,321778,345199,345439,324038,309872,316661,345017,327031,312660,334052,330381,328401,345371,301812,330001,305813,309419,335653,345389,345165
smape,0.057,0.266,0.163,0.06,0.058,0.068,0.08,0.056,0.058,0.028,0.045,0.041,0.155,0.076,0.017,0.206,0.12,0.126,0.142,0.143


In [23]:
df_submission_hist.index = pd.date_range(start=pd.Timestamp("2021-12-01") + pd.DateOffset(months=1), periods=26, freq='MS')
df_submission_hist

,321778,345199,345439,324038,309872,316661,345017,327031,312660,334052,330381,328401,345371,301812,330001,305813,309419,335653,345389,345165
2022-01-01,515.92,117.25,301.93,249.62,501.15,229.83,216.75,178.33,195.17,542.18,559.69,395.41,159.17,204.11,607.91,118.29,427.83,166.83,158.87,129.25
2022-02-01,515.96,117.25,301.89,249.61,501.28,229.79,216.90,178.31,195.16,542.15,559.67,395.46,159.19,204.11,607.94,118.37,427.93,166.85,159.21,129.33
2022-03-01,516.19,117.28,301.97,249.65,501.42,229.79,216.92,178.30,195.18,542.21,559.64,395.49,159.15,204.16,608.03,118.45,428.04,166.88,159.38,129.27
2022-04-01,516.19,117.23,302.04,249.64,501.57,229.68,216.91,178.25,195.21,542.47,559.86,395.44,159.13,204.42,608.14,118.46,427.93,166.93,159.34,129.25
2022-05-01,516.37,117.35,302.04,249.67,501.55,229.66,216.84,178.35,195.27,543.21,560.19,395.65,159.04,204.46,608.36,118.41,427.94,167.01,159.30,129.07
2022-06-01,516.46,117.44,302.19,249.68,501.71,229.76,216.79,178.38,195.19,543.67,560.64,395.92,159.08,204.41,608.53,118.38,428.21,167.03,159.20,128.89
2022-07-01,516.30,117.63,302.46,249.63,501.65,229.69,216.65,178.33,195.24,543.51,560.69,395.95,159.30,204.48,608.40,118.26,428.20,167.10,159.00,128.66
2022-08-01,516.34,117.53,302.56,249.64,501.57,229.61,216.31,178.38,195.23,543.29,560.67,395.87,159.43,204.56,608.21,118.14,428.14,167.13,158.99,128.63
2022-09-01,516.30,117.50,302.46,249.62,501.70,229.55,216.11,178.33,195.20,542.82,560.44,395.78,159.62,204.52,608.08,118.20,427.93,167.19,159.06,128.76
2022-10-01,516.16,117.45,302.50,249.61,501.54,229.52,216.24,178.26,195.19,542.70,560.29,395.75,159.66,204.54,608.09,118.21,427.77,167.21,159.06,128.77


## Catboost Model

In [ ]:
# one-hot encoding
transformer_exog = make_column_transformer(
    (
        OneHotEncoder(sparse_output=False, drop='if_binary'),
        make_column_selector(dtype_exclude=np.number),
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")


# Regressor hyperparameters search space
def search_space(trial):
  lags_grid = tuple([12, 24, [1, 2, 3, 4, 7, 9, 24]])
  search_space  = {
      'n_estimators'  : trial.suggest_int('n_estimators', 100, 1000, step=100),
      'max_depth'     : trial.suggest_int('max_depth', 3, 10, step=1),
      'learning_rate' : trial.suggest_float('learning_rate', 0.01, 1),
      'lags'          : trial.suggest_categorical('lags', lags_grid)
  }
  return search_space

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def search_hyperparameters(data, end_train, end_valid, exog_features,transformer_exog):

  # instantiate a forcaster transformer with categorical features
  forecaster = ForecasterAutoreg(
  regressor = CatBoostRegressor(
                  random_state=123,
                  silent=True,
                  allow_writing_files=False,
                  boosting_type = 'Plain', # Faster training
                  leaf_estimation_iterations = 3, # Faster training
              ),
  lags = 24,
  transformer_exog = transformer_exog
  )


  # search for best parameters
  results_search, frozen_trial = bayesian_search_forecaster(
  forecaster         = forecaster,
  y                  = data.loc[:end_valid, 'gw-level'],
  exog               = data.loc[:end_valid, exog_features],
  search_space       = search_space,
  steps              = 30,
  refit              = False,
  metric             = 'mean_absolute_percentage_error',
  initial_train_size = len(data.loc[:end_train]),
  fixed_train_size   = False,
  n_trials           = 20,
  random_state       = 123,
  return_best        = True,
  n_jobs             = 'auto',
  verbose            = False,
  show_progress      = True
  )

  best_params = results_search['params'].iat[0]

  return best_params

def train_and_predict(data, best_params, actual_data, end_valid, end_train, valid_num, train_num, df_idx, exog_features, transformer_exog, hrbnz01):

  # train for evaluation of the model
  forecaster = ForecasterAutoreg(
  regressor = CatBoostRegressor(**best_params,
                  random_state=123,
                  silent=True,
                  allow_writing_files=False,
                  boosting_type = 'Plain', # Faster training
                  leaf_estimation_iterations = 3, # Faster training
              ),
  lags = 24,
  transformer_exog = transformer_exog
  )

  # train the model the time series train and validation dataset
  forecaster.fit(
    y    = data.loc[:end_train, 'gw-level'],
    exog = data.loc[:end_train, exog_features]
  )

  # make predictions and evalute the model
  predictions = forecaster.predict(
      exog     = data.loc[df_idx[train_num+1]:, exog_features],
      steps    = 26
  )
  df_preds = pd.DataFrame(predictions)
  preds = df_preds["pred"].values
  # evaluate on symmetric mean absolute percentage error
  smape_value = smape(actual_data, preds)

  # train for future predictions
  forecaster = ForecasterAutoreg(
  regressor = CatBoostRegressor(**best_params,
                  random_state=123,
                  silent=True,
                  allow_writing_files=False,
                  boosting_type = 'Plain', # Faster training
                  leaf_estimation_iterations = 3, # Faster training
              ),
  lags = 24,
  transformer_exog = transformer_exog
  )

  # train the model the time series train and validation dataset
  forecaster.fit(
    y    = data.loc[:end_valid, 'gw-level'],
    exog = data.loc[:end_valid, exog_features]
  )

  # make predictions into the future
  predictions = forecaster.predict(
    exog     = data.loc[df_idx[valid_num+1]:, exog_features],
    steps    = 26
  )
  df_preds = pd.DataFrame(predictions)

  # free resources since it's going to run on iterations
  del forecaster

  return df_preds, smape_value

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
processed_data_dir = "/content/drive/MyDrive/processed_data_part1"
#test_template = "/content/drive/MyDrive/gw_test_empty.csv"

df_submission_cat, df_smape_cat = populate_test_data(processed_data_dir)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1, 2, 3, 4, 7, 9, 24] which is of type list.
  warnings.warn(message)


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
  Parameters: {'n_estimators': 200, 'max_depth': 4, 'learning_rate': 0.022161079642891046}
  Backtesting metric: 0.0025810083203441506



  5%|▌         | 1/20 [01:00<19:11, 60.60s/it]

...

  0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1, 2, 3, 4, 7, 9, 24] which is of type list.
  warnings.warn(message)


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12] 
  Parameters: {'n_estimators': 600, 'max_depth': 7, 'learning_rate': 0.1294223793304205}
  Backtesting metric: 0.0031559712752482947



100%|██████████| 20/20 [25:23<00:00, 76.15s/it]

> Done


In [ ]:
df_smape_cat.index = pd.Index(['smape'])
print(f" average smape: {round(df_smape_cat.mean(axis=1)[0],4)}")
df_smape_cat

 average smape: 0.1334


,300111,345710,329078,301838,345116,345512,326934,321646,335208,309625,309948,328104,310029,335778,326868,306167,345314,313569,313387,331124
smape,0.113,0.033,0.133,0.057,0.122,0.265,0.145,0.039,0.091,0.083,0.039,0.074,0.058,0.07,0.053,0.437,0.383,0.032,0.113,0.327


In [ ]:
df_submission_cat.index = pd.date_range(start=pd.Timestamp("2021-12-01") + pd.DateOffset(months=1), periods=26, freq='MS')
df_submission_cat

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,300111,345710,329078,301838,345116,345512,326934,321646,335208,309625,309948,328104,310029,335778,326868,306167,345314,313569,313387,331124
2022-01-01,153.95,399.98,237.86,200.80,224.41,230.92,242.37,510.71,328.32,492.39,638.37,424.66,658.52,251.61,283.84,113.23,120.91,181.15,144.01,140.20
2022-02-01,153.96,399.98,237.67,200.76,224.57,231.22,242.34,510.78,328.50,492.51,638.35,424.75,658.57,251.64,283.71,113.22,120.90,181.12,143.98,140.30
2022-03-01,153.96,399.99,237.74,200.89,224.75,230.99,242.31,510.91,328.54,493.06,638.42,424.71,658.63,251.67,283.93,113.51,121.04,181.18,144.10,140.54
2022-04-01,153.95,400.01,237.66,200.88,224.92,230.90,242.58,511.04,328.29,493.60,638.62,424.65,658.60,251.67,283.79,113.68,121.22,181.15,144.14,140.51
2022-05-01,153.94,400.02,237.59,200.80,224.82,230.80,243.16,511.04,328.06,493.54,638.84,424.73,658.60,251.73,283.93,113.69,121.45,181.14,144.11,140.60
2022-06-01,153.90,400.08,237.56,200.62,224.98,230.92,243.72,511.19,327.90,493.14,639.02,424.63,658.71,251.72,283.96,113.60,121.28,181.15,144.10,140.72
2022-07-01,153.93,400.13,237.41,200.56,225.12,230.91,243.68,511.09,328.11,492.95,639.02,424.71,658.81,251.73,283.96,113.48,121.12,181.25,144.03,140.63
2022-08-01,153.93,400.17,237.31,200.56,224.85,230.88,243.64,510.96,328.08,492.80,638.89,424.76,658.97,251.74,283.82,113.38,121.00,181.24,143.86,140.31
2022-09-01,153.93,400.15,237.08,200.68,224.61,230.69,243.57,510.93,328.49,492.83,638.82,424.73,659.04,251.68,283.78,113.22,120.93,181.22,143.93,140.23
2022-10-01,153.86,400.10,237.37,200.82,224.61,231.02,242.96,510.84,329.00,492.65,638.73,424.77,659.05,251.70,283.78,113.39,120.86,181.20,143.95,139.85


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## This is code for a single location prediction

In [ ]:
df_exog = pd.read_csv("/content/drive/MyDrive/processed_data/Burgenland-335588.csv")

# set categorical columns astype category for the transformer model to auto detect them
df_exog["season"] = df_exog["season"].astype("category")
df_exog["weather"] = df_exog["weather"].astype("category")
df_exog["date"] = pd.to_datetime(df_exog["date"])
df_exog.set_index("date", inplace=True)
# set the datatime range index to monthly freq
df_exog.index = pd.date_range(start=df_exog.index.min(), end=df_exog.index.max(), freq='MS')
df_exog

,gw-level,month,year,quarter,season,weather,month_sin,month_cos,quarter_sin,quarter_cos,...,poly_quarter_month_sin,poly_quarter_month_cos,poly_quarter_quarter_sin,poly_quarter_quarter_cos,poly_month_sin_month_cos,poly_month_sin_quarter_sin,poly_month_sin_quarter_cos,poly_month_cos_quarter_sin,poly_month_cos_quarter_cos,poly_quarter_sin_quarter_cos
1993-01-01,115.95,1,1993,1,winter,cold,2.588190e-01,0.965926,1.000000e+00,6.123234e-17,...,2.588190e-01,0.965926,1.000000e+00,6.123234e-17,2.500000e-01,2.588190e-01,1.584810e-17,9.659258e-01,5.914590e-17,6.123234e-17
1993-02-01,115.97,2,1993,1,winter,cold,5.000000e-01,0.866025,1.000000e+00,6.123234e-17,...,5.000000e-01,0.866025,1.000000e+00,6.123234e-17,4.330127e-01,5.000000e-01,3.061617e-17,8.660254e-01,5.302876e-17,6.123234e-17
1993-03-01,115.99,3,1993,1,spring,normal,7.071068e-01,0.707107,1.000000e+00,6.123234e-17,...,7.071068e-01,0.707107,1.000000e+00,6.123234e-17,5.000000e-01,7.071068e-01,4.329780e-17,7.071068e-01,4.329780e-17,6.123234e-17
1993-04-01,116.02,4,1993,2,spring,normal,8.660254e-01,0.500000,1.224647e-16,-1.000000e+00,...,1.732051e+00,1.000000,2.449294e-16,-2.000000e+00,4.330127e-01,1.060575e-16,-8.660254e-01,6.123234e-17,-5.000000e-01,-1.224647e-16
1993-05-01,116.02,5,1993,2,spring,normal,9.659258e-01,0.258819,1.224647e-16,-1.000000e+00,...,1.931852e+00,0.517638,2.449294e-16,-2.000000e+00,2.500000e-01,1.182918e-16,-9.659258e-01,3.169619e-17,-2.588190e-01,-1.224647e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-01,NaN,10,2023,4,Fall,normal,5.000000e-01,-0.866025,-2.449294e-16,1.000000e+00,...,2.000000e+00,-3.464102,-9.797174e-16,4.000000e+00,-4.330127e-01,-1.224647e-16,5.000000e-01,2.121150e-16,-8.660254e-01,-2.449294e-16
2023-11-01,NaN,11,2023,4,Fall,normal,2.588190e-01,-0.965926,-2.449294e-16,1.000000e+00,...,1.035276e+00,-3.863703,-9.797174e-16,4.000000e+00,-2.500000e-01,-6.339238e-17,2.588190e-01,2.365836e-16,-9.659258e-01,-2.449294e-16
2023-12-01,NaN,12,2023,4,winter,cold,1.224647e-16,-1.000000,-2.449294e-16,1.000000e+00,...,4.898587e-16,-4.000000,-9.797174e-16,4.000000e+00,-1.224647e-16,-2.999520e-32,1.224647e-16,2.449294e-16,-1.000000e+00,-2.449294e-16
2024-01-01,NaN,1,2024,1,winter,cold,2.588190e-01,0.965926,1.000000e+00,6.123234e-17,...,2.588190e-01,0.965926,1.000000e+00,6.123234e-17,2.500000e-01,2.588190e-01,1.584810e-17,9.659258e-01,5.914590e-17,6.123234e-17


In [ ]:
pipeline_categorical = make_pipeline(
                           OrdinalEncoder(
                               dtype=int,
                               handle_unknown="use_encoded_value",
                               unknown_value=-1,
                               encoded_missing_value=-1
                           ),
                           FunctionTransformer(
                               func=lambda x: x.astype('category'),
                               feature_names_out= 'one-to-one'
                           )
                       )

transformer_exog = make_column_transformer(
                       (
                           pipeline_categorical,
                           make_column_selector(dtype_exclude=np.number)
                       ),
                       remainder="passthrough",
                       verbose_feature_names_out=False,
                   ).set_output(transform="pandas")

# Lags grid
lags_grid = tuple([12, 24, [1, 2, 3, 4, 7, 9, 24]])

# Regressor hyperparameters search space
def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 400, 1200, step=100),
        'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 25, 500),
        'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 0.5),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1, step=0.1),
        'max_bin'         : trial.suggest_int('max_bin', 50, 250, step=25),
        'reg_alpha'       : trial.suggest_float('reg_alpha', 0, 1, step=0.1),
        'reg_lambda'      : trial.suggest_float('reg_lambda', 0, 1, step=0.1),
        'lags'            : trial.suggest_categorical('lags', lags_grid)
    }
    return search_space

In [ ]:
def search_hyperparameters(data, one_hot_encoder):

    # get the estimate end train and end validation dates
    exog_data = data.drop("gw-level", axis=1)
    exog_features = exog_data.columns
    df_idx = data.index
    train_num = int(len(data) * 0.8)
    end_train = "2012-12-01"
    end_valid = "2021-12-01"

    # instantiate a forcaster transformer with categorical features
    forecaster = ForecasterAutoreg(
        regressor = LGBMRegressor(random_state=15926, verbose=-1),
        lags = 24,
        transformer_exog = one_hot_encoder,
        fit_kwargs = {"categorical_feature": "auto"}
    )

    # search for best parameters
    results_search, frozen_trial = bayesian_search_forecaster(
    forecaster         = forecaster,
    y                  = data.loc[:end_valid, 'gw-level'],
    exog               = data.loc[:end_valid, exog_features],
    search_space       = search_space,
    steps              = 26,
    refit              = False,
    metric             = 'mean_absolute_percentage_error',
    initial_train_size = len(data.loc[:end_train]),
    fixed_train_size   = False,
    n_trials           = 20,
    random_state       = 123,
    return_best        = True,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
    )
    best_params = results_search['params'].iat[0]

    return best_params, forecaster

best_params, forecaster =  search_hyperparameters(df_exog, transformer_exog)

  0%|          | 0/20 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12] 
  Parameters: {'n_estimators': 1000, 'max_depth': 5, 'min_data_in_leaf': 132, 'learning_rate': 0.28014423685061673, 'feature_fraction': 0.9, 'max_bin': 125, 'reg_alpha': 1.0, 'reg_lambda': 0.7000000000000001}
  Backtesting metric: 0.0023255280123888103



/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1, 2, 3, 4, 7, 9, 24] which is of type list.
  warnings.warn(message)


In [ ]:
data = df_exog.copy()
exog_data = data.drop("gw-level", axis=1)
exog_features = exog_data.columns
df_idx = data.index
train_num = int(len(data) * 0.8)
valid_num = len(data.loc[:"2021-11-01"])
end_train = df_idx[train_num]
end_valid = df_idx[valid_num]
end_evaluation = df_idx[train_num+50]
evaluate_data = data.loc[df_idx[train_num+1]: end_evaluation, "gw-level"].values

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# train model with best params
forecaster = ForecasterAutoreg(
    regressor          = LGBMRegressor(**best_params, random_state=15926, verbose=-1),
    lags               = 24,
    transformer_exog   = transformer_exog,
    fit_kwargs         = {"categorical_feature": "auto"}
)
forecaster.fit(
    y    = data.loc[:end_train, 'gw-level'],
    exog = data.loc[:end_train, exog_features]
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# make predictions and evalute the model
predictions = forecaster.predict(
    exog     = data.loc[df_idx[train_num+1]:, exog_features],
    steps    = 50
)
pred_df = pd.DataFrame(predictions)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
preds = pred_df["pred"].values
smape_value = smape(evaluate_data, preds)
pred_df.loc["smape", "pred"] = smape_value
pred_df

,pred
2018-01-01 00:00:00,116.167604
2018-02-01 00:00:00,116.142285
2018-03-01 00:00:00,116.107357
2018-04-01 00:00:00,116.122247
2018-05-01 00:00:00,116.117663
2018-06-01 00:00:00,116.083430
2018-07-01 00:00:00,115.986986
2018-08-01 00:00:00,115.986986
2018-09-01 00:00:00,115.994564
2018-10-01 00:00:00,116.033680


In [ ]:
# make predictions into the future
predictions = forecaster.predict(
    exog     = data.loc[df_idx[valid_num+1]:, exog_features],
    steps    = 24
)
pd.DataFrame(predictions)

ValueError: To make predictions `exog` must start one step ahead of `last_window`.
    `last_window` ends at : 2017-12-01 00:00:00.
    `exog` starts at : 2022-01-01 00:00:00.
     Expected index : 2018-01-01 00:00:00.